# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [43]:
import twitter
import time
from collections import Counter
from prettytable import PrettyTable

#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'iEG3V3LLXNerRncmEWgv8lQ4n'
    CONSUMER_SECRET ='67HGDVQRF7YxJ6ZUFnUlvwreR2CShhao1Zx6k5wo0OoRRkGez4'
    OAUTH_TOKEN = '2380166015-sZywqhm3U9naQ154u0qQQzBv1FJgAWDVSWN6ROB'
    OAUTH_TOKEN_SECRET = '1AQCSvqp06DTpwYbgBXooWVmLuzWJ8zpsFpWgaazUhhha'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import json

# login to api
twitter_api = oauth_login()

# set initial search
search_results = twitter_api.search.tweets(q='Uber', count=100, lang='en')
statuses = search_results['statuses']
print("Length of statuses", len(statuses))
max_tweets = 1000

while(len(statuses) < max_tweets):
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    # search api will take the key words dictionary
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    print("Length of statuses", len(statuses))

# Write the status to a local file
outfile = open('tweets.txt', 'w')
outfile.write(json.dumps(statuses, indent=1))
outfile.close()
print "Tweets output to file successfully!"









('Length of statuses', 100)
('Length of statuses', 200)
('Length of statuses', 300)
('Length of statuses', 400)
('Length of statuses', 500)
('Length of statuses', 600)
('Length of statuses', 700)
('Length of statuses', 800)
('Length of statuses', 900)
('Length of statuses', 1000)
Tweets output to file successfully!


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [45]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect status texts
status_texts = [ status['text'] 
                 for status in statuses ]

# Use word to store all words in tweets and use counter to do analysis
word = []
for text in status_texts:
    for w in text.split():
        word.append(w)
word_freq = Counter(word)

# Put the data into the prettytable
top_word = PrettyTable(field_names=['Word','Count']) # Header
row_num = 0 # number of rows in table
for item in word_freq.most_common():
    if item[0] not in outlier:
        top_word.add_row(item) # Adding rows
        row_num += 1
    if row_num>30:
        break
top_word.align['Word'], top_word.align['Count'] = 'c', 'r' # Set column alignment
print top_word














+-------------------------+-------+
|           Word          | Count |
+-------------------------+-------+
|           code          |   920 |
|           ride          |   917 |
|           free          |   916 |
|           get           |   916 |
|          #Uber          |   911 |
|          promo          |   910 |
|          using          |   910 |
|           Use           |   909 |
|          taxis.         |   908 |
|            A            |   908 |
|          great          |   908 |
|         started         |   908 |
| https://t.co/iH1fjdXAgl |   908 |
|      @LoftyFollows:     |   908 |
|        vddrmu4yue       |   908 |
|          https…         |   908 |
|           way           |   908 |
|           uber          |    25 |
|           Tech          |    18 |
|          Uber,          |    17 |
|            To           |    12 |
|         Dropbox,        |    11 |
|          Other          |    11 |
|         Leaders         |    11 |
|          Boost          | 

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [46]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect all retweets
retweets = []
for status in statuses:
    if 'retweeted_status' in status:
        retweets.append((status['retweet_count'], status['retweeted_status']['user']['screen_name'], status['text']))

# Generate the pretty table
most_retweet = PrettyTable(field_names=['Count', 'Screen Name', 'Tweet Text']) # Header
for item in sorted(retweets, reverse=True)[:10]:
    most_retweet.add_row(item) # Adding rows
most_retweet.align['Count'], most_retweet.align['Screen Name'], most_retweet.align['Tweet Text'] = 'l', 'l', 'l'
most_retweet.max_width['Tweet Text'] = 50 # Set max width for "Tweet Text" column
print most_retweet











+-------+--------------+----------------------------------------------------+
| Count | Screen Name  | Tweet Text                                         |
+-------+--------------+----------------------------------------------------+
| 2004  | LoftyFollows | RT @LoftyFollows: A free #Uber ride is a great way |
|       |              | to get started using Uber taxis. Use promo code    |
|       |              | vddrmu4yue https://t.co/iH1fjdXAgl https…          |
| 2004  | LoftyFollows | RT @LoftyFollows: A free #Uber ride is a great way |
|       |              | to get started using Uber taxis. Use promo code    |
|       |              | vddrmu4yue https://t.co/iH1fjdXAgl https…          |
| 2004  | LoftyFollows | RT @LoftyFollows: A free #Uber ride is a great way |
|       |              | to get started using Uber taxis. Use promo code    |
|       |              | vddrmu4yue https://t.co/iH1fjdXAgl https…          |
| 2004  | LoftyFollows | RT @LoftyFollows: A free #Uber ride is 

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [47]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect hashtags
hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Collect mention user
mention_user = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

# Generate pretty table
for label, data in (('Hashtag',hashtags), ('Mention User', mention_user)): # Access two table data
    table = PrettyTable(field_names=[label, 'Count']) # Header
    count = Counter(data) # Count the data
    for item in count.most_common()[:10]:
        table.add_row(item) # Adding each row
    table.align[label], table.align['Count'] = 'l', 'c'
    print table






+------------+-------+
| Hashtag    | Count |
+------------+-------+
| Uber       |  913  |
| uber       |   3   |
| infosec    |   3   |
| Boston     |   2   |
| security   |   2   |
| birtherism |   1   |
| apple      |   1   |
| transit    |   1   |
| LittleCab  |   1   |
| freeapp    |   1   |
+------------+-------+
+----------------+-------+
| Mention User   | Count |
+----------------+-------+
| LoftyFollows   |  908  |
| Arzaylea       |   5   |
| lindsaydemeola |   5   |
| Uber_India     |   3   |
| Uber           |   3   |
| undeux         |   2   |
| Uber_CO        |   2   |
| ChiefKeef      |   2   |
| RedskiHusky    |   1   |
| odavis_        |   1   |
+----------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [33]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Get all friends
friends = twitter_api.friends.ids(screen_name='AutonomousCaRR', count=5000)
friend_list = friends['ids']
next_cursor = friends['next_cursor']
while (next_cursor != 0 and len(friend_list) < 5000*15): # Navigate through pages by cursor and rate controller
    friends = twitter_api.friends.ids(screen_name='AutonomousCaRR', cursor=next_cursor, count=5000)
    friend_list += friends['ids']
    next_cursor = friends['next_cursor']


# Get all followers
followers = twitter_api.followers.ids(screen_name='AutonomousCaRR', count=5000)
follower_list = followers['ids']
next_cursor = followers['next_cursor']
rate_counter = 1
while (next_cursor != 0 and len(follower_list) < 5000*15): # Navigate through pages by cursor and rate controller
    followers = twitter_api.followers.ids(screen_name='AutonomousCaRR', cursor=next_cursor, count=5000)
    follower_list += followers['ids']
    next_cursor = followers['next_cursor']




In [34]:
# Create table for friend
friend_table = PrettyTable(field_names=['Friend ID', 'Screen Name']) # Header
for user in twitter_api.users.lookup(user_id=friend_list[:20]):
    friend_table.add_row([user['id'], user['screen_name']])
friend_table.align['Friend ID'], friend_table.align['Screen Name'] = 'l', 'c' # Alignment
print friend_table

# Create table for follower
follower_table = PrettyTable(field_names=['Follower ID', 'Screen Name']) # Header
for user in twitter_api.users.lookup(user_id=follower_list[:20]):
    follower_table.add_row([user['id'], user['screen_name']])
follower_table.align['Follower ID'], follower_table.align['Screen Name'] = 'l', 'c' # Alignment
print follower_table

+------------+-----------------+
| Friend ID  |   Screen Name   |
+------------+-----------------+
| 40796368   |     kaybed1     |
| 223514649  |   Zeyawinhtut   |
| 292825731  |     smkayes     |
| 113802609  |  RachaelAnne_3  |
| 104746157  |    Sorafatul    |
| 235160347  |     rbenlee     |
| 1669662031 | saikatsamanta18 |
| 52056156   |    Mwardi300    |
| 465888120  |     HP101201    |
| 473720203  |   ferryantonie  |
| 159912065  |  DataRecoverer  |
| 411747568  |  heavydutyrigs  |
| 136652675  |     mwahyyd     |
| 1094487468 |  NichakornBogy  |
| 419777782  |  alamashraf2020 |
| 302055316  |     anjan_91    |
| 111618328  |  UMANGLOVESLIFE |
| 275031863  |  JenBDesjardins |
+------------+-----------------+
+--------------------+-----------------+
| Follower ID        |   Screen Name   |
+--------------------+-----------------+
| 776761808577331200 |  judytheredhead |
| 4806578173         |   AmparoKenner  |
| 776550863431200769 |   ChAs69672343  |
| 773913354192707586 |    vi

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [41]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Create Set for friend and follower
friend = set(friend_list)
follower = set(follower_list)

# Check mutual friend
mutual_friend = friend.intersection(follower)
mutual_friend = list(mutual_friend)
if (len(mutual_friend) == 0):
    print "There are no mutual friends."
else:
    mutual_table = PrettyTable(field_names=['Mutual Friend ID', 'Screen Name']) # Create table for mutual friend
    for mutual in twitter_api.users.lookup(user_id=mutual_friend[:30]):
        mutual_table.add_row([mutual['id'], mutual['screen_name']])
    mutual_table.align['Mutual Friend ID'], mutual_table.align['Screen Name'] = 'l', 'c'
    print mutual_table






+------------------+-----------------+
| Mutual Friend ID |   Screen Name   |
+------------------+-----------------+
| 857645059        |  AutomotivePRBE |
| 1256669196       |    wiemsoukeh   |
| 998318094        |      tc604      |
| 2338316309       |   FloreaMatei   |
| 2849882135       |  XclusiveAutoNJ |
| 2692718616       | MrDWilliamsBGHS |
| 353665049        |      Biltac     |
| 21495840         |     gr8n8no     |
| 19578913         |      Verio      |
| 2373877795       |  Ancaster_Group |
| 256655396        |   orilayeelay   |
| 35545125         |    FresnoBill   |
| 54575144         |    carlisence   |
| 378019885        |    KevTynan10   |
| 2754117678       |    avtomaser    |
| 204955698        |   jacobyaaron   |
| 48906292         |     c230mike    |
| 648543           |     gseifert    |
| 546054204        |  AndreaRapelli  |
| 237846600        |  Ken_Simmons_NL |
| 214237258        |    xaviroca1    |
| 268247115        |  TrendLineNews  |
| 135290960        |   Wa

*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [1]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import twitter
import json
import pymongo
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'iEG3V3LLXNerRncmEWgv8lQ4n'
    CONSUMER_SECRET ='67HGDVQRF7YxJ6ZUFnUlvwreR2CShhao1Zx6k5wo0OoRRkGez4'
    OAUTH_TOKEN = '2380166015-sZywqhm3U9naQ154u0qQQzBv1FJgAWDVSWN6ROB'
    OAUTH_TOKEN_SECRET = '1AQCSvqp06DTpwYbgBXooWVmLuzWJ8zpsFpWgaazUhhha'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

def mongoDB_connect():
    client = pymongo.MongoClient()
    return client




In [2]:
# login to api
twitter_api = oauth_login()
# set up database
client = mongoDB_connect()
db = client.ds501case1
collection = db.tweets

In [97]:
# main process to connect database and collect new data
def collecte_DB(max_tweets, key_word, clean_history):
    # parameter initial
    index = 0
    statuses = []
    
    # clean history data
    if clean_history:
        collection.delete_many({})
        print 'History Data are cleaned'

    for index in range(len(key_word)):
        # initial search
        word = key_word[index]
        search_results = twitter_api.search.tweets(q=word, count=100, lang='en')
        statuses += search_results['statuses']

        # continue get result
        while(len(statuses) < (max_tweets*(index+1))):
            try:
                next_results = search_results['search_metadata']['next_results']
            except KeyError: # No more results when next_results doesn't exist
                break

            # Create a dictionary from next_results, which has the following form:
            # ?max_id=313519052523986943&q=NCAA&include_entities=1
            kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])

            # search api will take the key words dictionary
            search_results = twitter_api.search.tweets(**kwargs)
            statuses += search_results['statuses']
    print "Final volume of problem 4 tweets is %d" % len(statuses)

    # write data into mongoDB
    for status in statuses:
        res = collection.insert_one(status)
    print "Problem 4 database is generated successfully!"

    
# set initial parameters
max_tweets = 1000
key_word = ['driverless',
            '‪AutonomousCaRR‪',
            'googlecar',
            'autonomouscars',
            'selfdriving',
            'futuristiccars',
            'driverlesstechnology',
            'SelfDrivingUber',
            'UGV']

# collect data and write into DB, enable this line iff new tweet data is necessary
#collecte_DB(max_tweets, key_word, True)



History Data are cleaned
Final volume of problem 4 tweets is 5754
Problem 4 database is generated successfully!


## Sematic Orientation

In [133]:
import pandas as pd
import nltk
import re
import string
from collections import defaultdict
from collections import Counter
import numpy as np



# load words pool from Bing Liu
pos_pool = pd.read_table('positive-words.txt')
neg_pool = pd.read_table('negative-words.txt')
pos_pool.columns = ['word']
neg_pool.columns = ['word']
pos_pool = pos_pool['word'].tolist() 
neg_pool = neg_pool['word'].tolist()



# function to extract words inside the tweet
def extract_word(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    words = [e.lower() for e in nltk.tokenize.word_tokenize(tweet) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    return words



# build word counter, co-occurance matrix
com = defaultdict(lambda: defaultdict(int))
words_pool = []

for tweet in collection.find({}):
    text = extract_word(tweet['text']) # extract words
    
    words_pool += text # build words pool
    
    for i in range(len(text)-1): # build co-occurance matrix
        for j in range(i+1, len(text)):
            w1, w2 = sorted([text[i], text[j]])
            if w1 != w2:
                com[w1][w2] += 1

words_counter = Counter(words_pool)



# compute word prob
p_w = {}
p_com = defaultdict(lambda : defaultdict(int))
n_pool = float(len(words_pool))
 
for word, n in words_counter.items():
    p_w[word] = n / n_pool
    for cow in com[word]:
        p_com[word][cow] = com[word][cow] / n_pool

        
               
# compute pmi and semantic orientation
pmi = defaultdict(lambda : defaultdict(int))
for w1 in p_w:
    for w2 in com[w1]:
        denom = p_w[w1] * p_w[w2]
        pmi[w1][w2] = np.log2(p_com[w1][w2] / denom)

SO_m = {}
for word, n in p_w.items():
    positive_assoc = sum(pmi[word][tx] for tx in pos_pool)
    negative_assoc = sum(pmi[word][tx] for tx in neg_pool)
    SO_m[word] = positive_assoc - negative_assoc


In [134]:
import operator



# explore the top positive and negative words
semantic_sorted = sorted(SO_m.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
top_pos = semantic_sorted[:10]
top_neg = semantic_sorted[-10:]
 
print(top_pos)
print(top_neg)

[(u'autonomous', 102.59599728653649), (u'day', 74.592393605285181), (u'auto', 67.197953748683531), (u'car', 63.089132262226912), (u'amp', 61.319966145899798), (u'ford', 58.562826182813524), (u'a-ugv', 58.148538210774625), (u'cross', 53.627754539585183), (u'autos', 52.483989810594252), (u'going', 51.839117246380738)]
[(u'cops', -39.357584337037551), (u'chrisl2185', -40.303136553015172), (u'idea', -41.601646554888838), (u'piss', -42.058024055178635), (u'also', -45.667216476520686), (u'freedom', -46.058024055178635), (u'bad', -46.203932532387775), (u'insurance', -52.599517040311937), (u'could', -55.900193462236757), (u'apple', -56.133465750064509)]


## Sentiment Classifier

In [82]:
import pandas as pd
import nltk
import re
import string

# read in training data for NLP, this training data is downloaded from kaggle.com for tweet sentiment analysis
nlp_train = pd.read_table('training.txt')

# split train words
train_data = []
stop = nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['RT', 'rt', 'via']

def extract_train_word(x, train_data):
    words = [e.lower() for e in nltk.tokenize.word_tokenize(x['Tweet']) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    sentiment = x['Sentiment']
    train_data.append((words, sentiment))
    
result = nlp_train.apply(lambda x: extract_train_word(x, train_data), axis=1)

# load words pool from Bing Liu
pos_pool = pd.read_table('positive-words.txt')
neg_pool = pd.read_table('negative-words.txt')
pos_pool.columns = ['word']
neg_pool.columns = ['word']
word_pool = pos_pool['word'].tolist() + neg_pool['word'].tolist()

# feature extraction with words pool
def feature_extract(tweets):
    global word_pool
    features = {}
    dry_tweet = set(tweets)
    for word in word_pool:
        features['Contain (%s)'%word] = (word in dry_tweet)
    return features

# generate training set
train_set = nltk.classify.apply_features(feature_extract, train_data)

# train classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)



/home/sylor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [93]:
# function to extract words inside the tweet
def extract_word(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    words = [e.lower() for e in nltk.tokenize.word_tokenize(tweet) 
             if ((len(e) >= 3) and (e not in stop) and (not e.startswith(('#','@'))))]
    return words


# test classify
result_board = ['negative', 'positive']

test_tweet = 'driveless car is unsafe'
res = classifier.classify(feature_extract(extract_word(test_tweet)))
print 'The tweet \'%s\' is %s' % (test_tweet, result_board[res])


The tweet 'driveless car is unsafe' is negative


In [102]:
tweet = collection.find_and_modify({'retweet_count':{'$gte':500}})
text = extract_word(tweet['text'])
label = result_board[classifier.classify(feature_extract(text))]
print tweet['text']
print label

RT @swaaanson: Musk's greatest gift to humanity will not be driverless cars or missions to Mars. It will be inspiring a whole generation to…
negative


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

